In [2]:
import bz2
import json
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd

from six.moves import urllib
from time import time
from tqdm import tqdm


In [ ]:
xmlfile='idwiki-20230720-pages-articles-multistream.xml'
ns = {'export-0.1': 'http://www.mediawiki.org/xml/export-0.10/'}
tags_to_skip = ["siteinfo"]


import xml.etree.ElementTree as ET

def parse_wiki_xml(filename):
    # tags_to_skip는 건너뛸 태그의 목록입니다. 예를 들어, "siteinfo" 태그 등을 건너뛰는 용도로 사용됩니다.
    skipping = ""  # 현재 건너뛰고 있는 태그를 저장하는 변수
    in_page = False  # 현재 'page' 태그 내부에 있는지 여부를 나타내는 변수

    # XML 파일을 파싱하면서 start와 end 이벤트를 반복적으로 처리합니다.
    for event, elem in ET.iterparse(filename, events=("start", "end",)):
        if event == "start":
            # tags_to_skip에 해당하는 태그가 있다면, 해당 태그를 건너뜁니다.
            for tag in tags_to_skip:
                if tag in elem.tag:
                    print("removing elem siteinfo")  # 현재 siteinfo 태그를 건너뛰고 있다는 메시지를 출력합니다.
                    skipping = tag
                    elem.clear()  # 건너뛰는 태그를 제거합니다.
                    break
            if in_page:
                continue
            if "page" in elem.tag:
                in_page = True  # 'page' 태그 내부에 들어왔으므로 in_page를 True로 설정합니다.
        elif event == "end":
            if skipping:
                # 현재 건너뛰고 있는 태그를 처리합니다.
                if skipping in elem.tag:
                    elem.clear()  # 건너뛰는 태그를 제거합니다.
                    skipping = ""  # skipping 변수를 초기화합니다.
            else:
                # 건너뛰는 태그가 아니라면 'page' 태그를 처리합니다.
                if "page" in elem.tag:
                    yield elem  # 'page' 태그를 반환합니다.
                    elem.clear()  # 'page' 태그를 처리했으므로 해당 태그를 제거합니다.
                    in_page = False  # 'page' 태그 내부를 빠져나왔으므로 in_page를 False로 설정합니다.
                    

pages = parse_wiki_xml(xmlfile)

def process_text(text):
    # Remove any text not normally visible
    text = re.sub(r"&amp;", "&", text)  # decode URL encoded chars
    text = re.sub(r"&nbsp;", " ", text) #html에서의 공백 제거
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<.*>", "", text)  # remove xml tags
    text = re.sub(r"<ref[^<]*</ref>", "", text)  # remove references <ref...> ... </ref>
    text = re.sub(r"<[^>]*>", "", text)  # remove xhtml tags
    text = re.sub(r"\[http:[^] ]*", "[", text)  # remove normal url, preserve visible text
    text = re.sub(r"\|thumb", "", text)  # remove images links, preserve caption
    text = re.sub(r"\|left", "", text)
    text = re.sub(r"\|right", "", text)
    text = re.sub(r"\|\d+px", "", text)
    text = re.sub(r"\[\[image:[^\[\]]*\|", "", text)
    text = re.sub(r"\[\[category:([^|\]]*)[^]]*\]\]", r"\1", text, flags=re.I)  # show categories without markup
    text = re.sub(r"\[\[[a-z\-]*:[^\]]*\]\]", "", text)  # remove links to other languages
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)  # remove wiki url, preserve visible text
    text = re.sub(r"{{[^}]*}}", "", text)  # remove {{icons}} and {tables}
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\[", "", text)  # remove [ and ]
    text = re.sub(r"\]", "", text)
    text = re.sub(r"\(", "", text)  # remove ( and )
    text = re.sub(r"\)", "", text)
    text = re.sub(r"&[^;]*;", " ", text)  # remove URL encoded chars
    text = re.sub(r"\"", "", text)  # remove ' and "
    text = re.sub(r"'", "", text)
    text = re.sub(r"_", "", text)  # remove _
    text = re.sub(r"\W+", " ", text)
    return text

def convert_to_df(pages):
    data = []
    t0 = time()
    for i, page in enumerate(pages):
        if i % 2000 == 1999:
            print("Read {}k articles. Elapsed time: {:.3f}s".format(int((i+1)/1000), time() - t0), end="\r")

        title = page.find('export-0.1:title', ns).text.lower()
        
        text = page.find('export-0.1:revision', ns).find('export-0.1:text', ns).text
        if not text:
            page.clear()
            continue

        text = process_text(text)

        words = text.split()
        
        total_words = len(words)
        total_long_words = len([w for w in words if len(w) > 3])
        if total_long_words < 15:
            page.clear()
            continue
        date = page.find('export-0.1:revision', ns).find('export-0.1:timestamp', ns).text
        text =  " ".join(words)
        data.append([title, date, text])

    # Membuat data frame dari list data
    columns = ['title', 'date', 'text']
    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
# XML 파일 파싱
pages = parse_wiki_xml(xmlfile)

# 파싱된 페이지를 데이터 프레임으로 변환
df = convert_to_df(pages)

# 위키미디어 데이터를 JSON 형식으로 변환하여 텍스트 파일로 저장
text_filename = os.path.splitext(xmlfile)[0] + ".txt"
print("Start writing Wikipedia texts to {}".format(text_filename))
t0 = time()

with open(text_filename, "wb") as f:
    for i, page in enumerate(pages):
        text = process_text(page.find('export-0.1:revision', ns).find('export-0.1:text', ns).text)
        f.write((json.dumps({"text": text}) + os.linesep).encode("utf-8"))
        if i % 1000 == 999:
            print("Done writing {}k pages. Elapsed time: {:.3f}s".format(int((i+1)/1000), time() - t0), end="\r")
print("Done writing Wikipedia texts in {:.3f}s".format(time() - t0))